In [1]:
import cv2
import torch
from Encoding import *

In [2]:
img = cv2.imread('lena1.tif')
img = cv2.resize(img, (4, 4))
img = torch.from_numpy(img)
img = img.sum(2)//3
# ttfs = TimeToFirstSpikeEncoding(img, 100, 400)
# spikes = ttfs.encode()
# for i in range(spikes.shape[0]):
#     for j in range(spikes.shape[1]):
#         print(spikes[i][j].item(), end=" ")
#     print()

ge = PoissonEncoder(img, 10)
spikes = ge.encode()
# ge = PositionEncoder(img, 10, 3)
# spikes = ge.encode()
# ge.encode()
print(spikes)
# ge = GaussianEncoding(10, 100, img)
# ge.encode()


tensor([[0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.],
        [0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.]])
